# 2016 Bot 

### Overview
- Introduction 
- Data ETL
- Data Augmentation 
- Networks Architecture 
- Training 
- Model Averaging 
- Conclusion 

In [35]:
import os, sys, glob
import numpy as np
import cv2
from skimage.io import imread
from sklearn.utils import shuffle

In [38]:
from keras.preprocessing.image import ImageDataGenerator


from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import model_from_json


In [33]:
class Conf:
    def __init__(self, confPath):
        # load and store the configuration and update the object's dictionary
        conf = json.loads(open(confPath).read())
        self.__dict__.update(conf)

    def __getitem__(self, k):
        # return the value associated with the supplied key
        return self.__dict__.get(k, None)
    
    
def auto_resized(img,size):
    '''size = (width,height)'''
    size = tuple(size)
    resize_img = cv2.resize(img, size, interpolation=cv2.INTER_LINEAR)
    return resize_img

def TrainFilePath(folderPath, constrain=None, **kargs):
    '''
    (1) Output filepath and calssName
    (2) folderPath 
          --label_1
           -- xxx.jpg
    '''
    assert folderPath[-1]!='/'
    if constrain is None:
        constrain = ('avi', 'mp4','png','jpg') 
    for (rootDir, dirNames, fileNames) in os.walk(folderPath):
        for fileName in fileNames:
            if fileName.split('.')[-1] in constrain:
                yield (os.path.join(rootDir, fileName)) 
                
#img_channels = 3
def genTrX(filePath, resolution, img_channels=3):
    assert type(resolution) == tuple
    img = auto_resized(imread(filePath),resolution)  #conf['sliding_size']
    if img_channels==1:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    elif img_channels==3:
        img = img[:,:,:3]
    return img
                
def load_training(folderList, img_rows, img_cols, img_channels):
    TrY = []
    TrX = []
    TrY_template = np.eye(len(folderList))
    for eyeId, folderPath in enumerate(folderList):
        for imgPath in TrainFilePath(folderPath) :
            TrY.append(TrY_template[eyeId])
            TrX.append(genTrX(imgPath, (img_rows,img_cols), img_channels))
    print (len(TrX))
    return TrX, TrY

def create_folderList(rootDir):
    result=[]
    for a in os.listdir(rootDir):
        a = os.path.join(rootDir, a)
        if os.path.isdir(a):
            result.append(a) 
    return result


def reshapeShuffle(TrX, TrY, img_rows, img_cols, img_channels):
    trainX = np.asarray(TrX, dtype = np.uint8)
    trainX = trainX.reshape(trainX.shape[0], img_channels, img_rows, img_cols)
    trainX = trainX.astype('float32')
    trainY = np.asarray(TrY, dtype = np.float32)
    # shuffle
    trainX , trainY = shuffle(trainX,trainY)
    print ('Train_X : ',trainX.shape,'Train_Y' ,trainY.shape)
    return trainX , trainY

In [30]:
ROOT_Dir = 'D:\\2016bot_cv'

img_rows= 48

img_cols= 48

img_channels=3

folderList = create_folderList(ROOT_Dir)

Train_X, Train_Y = load_training(folderList, img_rows, img_cols, img_channels)



#TrainFilePath('D:\2015')

116481


In [36]:
train_X , train_Y = reshapeShuffle(Train_X, Train_Y, img_rows, img_cols, img_channels=img_channels)

('Train_X : ', (116481L, 3L, 48L, 48L), 'Train_Y', (116481L, 12L))


In [37]:

class ConvNetFactory:
	def __init__(self):
		pass

	@staticmethod
	def build(name, *args, **kargs):
		# define the network (i.e., string => function) mappings
		mappings = {
			"shallownet": ConvNetFactory.ShallowNet,
			"lenet": ConvNetFactory.LeNet,
			"karpathynet": ConvNetFactory.KarpathyNet,
			"minivggnet": ConvNetFactory.MiniVGGNet}

		# grab the builder function from the mappings dictionary
		builder = mappings.get(name, None)

		# if the builder is None, then there is not a function that can be used
		# to build to the network, so return None
		if builder is None:
			return None

		# otherwise, build the network architecture
		return builder(*args, **kargs)

	@staticmethod
	def ShallowNet(numChannels, imgRows, imgCols, numClasses, **kwargs):
		# initialzie the model
		model = Sequential()

		# define the first (and only) CONV => RELU layer
		model.add(Convolution2D(32, 3, 3, border_mode="same",
			input_shape=(numChannels, imgRows, imgCols)))
		model.add(Activation("relu"))

		# add a FC layer followed by the soft-max classifier
		model.add(Flatten())
		model.add(Dense(numClasses))
		model.add(Activation("softmax"))

		# return the network architecture
		return model

	@staticmethod
	def LeNet(numChannels, imgRows, imgCols, numClasses, activation="tanh", **kwargs):
		# initialize the model
		model = Sequential()

		# define the first set of CONV => ACTIVATION => POOL layers
		model.add(Convolution2D(20, 5, 5, border_mode="same",
			input_shape=(numChannels, imgRows, imgCols)))
		model.add(Activation(activation))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# define the second set of CONV => ACTIVATION => POOL layers
		model.add(Convolution2D(50, 5, 5, border_mode="same"))
		model.add(Activation(activation))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# define the first FC => ACTIVATION layers
		model.add(Flatten())
		model.add(Dense(500))
		model.add(Activation(activation))

		# define the second FC layer
		model.add(Dense(numClasses))

		# lastly, define the soft-max classifier
		model.add(Activation("softmax"))

		# return the network architecture
		return model

	@staticmethod
	def KarpathyNet():
		# initialize the model
		model = Sequential()

		# define the first set of CONV => RELU => POOL layers
		model.add(Convolution2D(16, 5, 5, border_mode="same",
			input_shape=(numChannels, imgRows, imgCols)))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# check to see if dropout should be applied to reduce overfitting
		if dropout:
			model.add(Dropout(0.25))

		# define the second set of CONV => RELU => POOL layers
		model.add(Convolution2D(20, 5, 5, border_mode="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# check to see if dropout should be applied to reduce overfitting
		if dropout:
			model.add(Dropout(0.25))

		# define the third set of CONV => RELU => POOL layers
		model.add(Convolution2D(20, 5, 5, border_mode="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

		# check to see if dropout should be applied to reduce overfitting
		if dropout:
			model.add(Dropout(0.5))

		# define the soft-max classifier
		model.add(Flatten())
		model.add(Dense(numClasses))
		model.add(Activation("softmax"))

		# return the network architecture
		return model

	@staticmethod
	def MiniVGGNet():
		# initialize the model
		model = Sequential()

		# define the first set of CONV => RELU => CONV => RELU => POOL layers
		model.add(Convolution2D(32, 3, 3, border_mode="same",
			input_shape=(numChannels, imgRows, imgCols)))
		model.add(Activation("relu"))
		model.add(Convolution2D(32, 3, 3))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2)))

		# check to see if dropout should be applied to reduce overfitting
		if dropout:
			model.add(Dropout(0.25))

		# define the second set of CONV => RELU => CONV => RELU => POOL layers
		model.add(Convolution2D(64, 3, 3, border_mode="same"))
		model.add(Activation("relu"))
		model.add(Convolution2D(64, 3, 3))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2)))

		# check to see if dropout should be applied to reduce overfitting
		if dropout:
			model.add(Dropout(0.25))

		# define the set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation("relu"))

		# check to see if dropout should be applied to reduce overfitting
		if dropout:
			model.add(Dropout(0.5))

		# define the soft-max classifier
		model.add(Dense(numClasses))
		model.add(Activation("softmax"))

		# return the network architecture
		return model

In [39]:
gen_Img = ImageDataGenerator(featurewise_center=False,
    samplewise_center=True,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=True,
    rotation_range=180.,
    width_shift_range=5.,
    height_shift_range=5.,
    shear_range=10.,
    zoom_range=10.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=False,
    vertical_flip=False,
    rescale=None,
    dim_ordering=K.image_dim_ordering())


NameError: name 'K' is not defined